<a href="https://colab.research.google.com/github/amontenegrot/COVID19/blob/main/COVID19_SIR_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar datos

Datos oficiales:

[Mundo](https://ourworldindata.org/coronavirus-source-data)

[Colombia](https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx)

##Librerias

Lectura de datos y ajustes de visualización

In [6]:
#%load_ext google.colab.data_table #Visualizar el DataFrame de forma más eficiente en la pantalla

In [7]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots

dfWorld = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/total_cases.csv')
dfMuertes = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/total_deaths.csv')

#Modelo SIR

Modelo estadístico que pretende describir la propagación de determinada enfermedad en una población específica. En ese sentido, el modelo contempla tres clasificaciones en relación al tiempo:

$t$: tiempo \\
$N$: población

$S(t)$: personas suceptibles a enfermarse (no han sido contagiadas) \\
$I(t)$: personas infectadas \\
$R(t)$: personas recuperadas (inmunes)

$β$: tasa de transmisión \\
$γ$: tasa de recuperación

##Parametros
$N$: 48.258.494 \\
$β$: 2.2 \\
$γ$: 1/5.2

##Condiciones iniciales

$R(0)$: 0 \\
$I(1)$: 1

In [8]:
N =  48258494 #Población total (Colombia)

I0 = 1 #Valores iniciales de personas Infectadas (I0)
R0 = 0 #Valores iniciales de personas que se recuperaron (R0)

#El resto de la población, S0 son las personas que están sujetos a infección inicialmente.
S0 = N - I0 - R0

# Tasa de transmisión, tasa de recuperación en 1/días
beta = 2.2
gamma = 1/5.2

In [9]:
#Puntos en la gráfica (en días)
t = np.linspace(0, 365, 365)

In [10]:
#Las ecuaciones diferenciales del modelo SIR
def deriv(y, t, N, beta, gamma):
   S, I, R = y
   dSdt = -beta * S * I / N
   dIdt = beta * S * I / N - gamma * I
   dRdt = gamma * I
   return dSdt, dIdt, dRdt

In [11]:
#Vector de las condiciones iniciales
y0 = S0, I0, R0

#Resolver el sistema de ecuaciones diferenciales, en la secuencia de días que ya definimos
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

In [12]:
#Plot the data on three separate curves for S(t), I(t) and R(t)
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=I,mode='lines',name='Infected'))
fig.show()

In [13]:
valores = [(2.2,1/2.3), (5,1/7), (1,1/3)]

def modelo(beta, gamma):
  N =  48258494 #Población total (Colombia)
  I0 = 1 #Valores iniciales de personas Infectadas (I0)
  R0 = 0 #Valores iniciales de personas que se recuperaron (R0)

  #El resto de la población, S0 son las personas que están sujetos a infección inicialmente.
  S0 = N - I0 - R0

  #Puntos en la gráfica (en días)
  t = np.linspace(0, 365, 365)

  #Las ecuaciones diferenciales del modelo SIR
  def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

  #Vector de las condiciones iniciales
  y0 = S0, I0, R0

  #Resolver el sistema de ecuaciones diferenciales, en la secuencia de días que ya definimos
  ret = odeint(deriv, y0, t, args=(N, beta, gamma))
  S, I, R = ret.T
  return(I)

#Para la parte de las gráficas:
py.iplot([{
   'x': t,
   'y': modelo(*valor),
   'name': str(valor),
}  for valor in valores], filename='cufflinks/multiple-lines-on-same-chart')  